                                             Housing-price-predictor


In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
housing = pd.read_csv("data.csv")
#housing.head(), #housing.info(), #housing['CHAS'].value_counts(),#housing.describe()

In [ ]:

#housing.hist(bins = 50, figsize=(20,15))

Train-test splitting

In [ ]:

def split_train_test(data, test_ratio):
    np.random.seed(42)
    shuffled = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled[:test_set_size]
    train_indices = shuffled[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
#train_set, test_set = split_train_test(housing, 0.2)

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state =42)


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits = 1, test_size= 0.2, random_state = 42)
for train_index, test_index in split.split(housing,housing['CHAS']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
housing = strat_train_set.copy()

##Looking for correlations

In [ ]:
corr_matrix = housing.corr()
corr_matrix['MEDV'].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["RM","ZN","MEDV","LSTAT"]
scatter_matrix(housing[attributes],figsize=(12,8))

In [ ]:
housing.plot(kind="scatter",x="RM", y = "MEDV",alpha = 0.8)

##Trying out different attributes

In [ ]:
housing["TPM"] = housing['TAX']/housing['RM']

In [ ]:
housing.head()

In [ ]:
corr_matrix = housing.corr()
corr_matrix['MEDV'].sort_values(ascending=False)

In [ ]:
housing.plot(kind="scatter",x="TPM", y = "MEDV",alpha = 0.8)

##Missing attributes

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy="median")
imputer.fit(housing)


##Scikit-learn Design
#primarly, three types of objects
1. Estimators - It estimates some parameters based on a dataset.
2. Transformers - Transform method takes input and returns uptput based on the learning from fit()
3. Predictors - Linear Regression model is an example of predictor. fit() and predict() are two common functions. It also gives score function which will evaluate the predictions.